# Basic Setup

Select whether to use Google Colab or not.

In [1]:
# SET HERE if notebook gets executed on google colab or locally
is_on_colab = False

In [2]:
if is_on_colab:
    # Google Colab setup
    from google.colab import drive
    drive.mount('/content/drive')

    import os
    os.chdir("/content/drive/My Drive/Novel_View_Synthesis")

In [3]:
# ONLY NECESSARY FOR LOCAL EXECUTION (WORKS WITHOUT THIS CELL IN GOOGLE COLAB)
# Setup that is necessary for jupyter notebook to find sibling-directories
# see: https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im


if not is_on_colab:
    
    import os
    import sys
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)


In [4]:
# Imports for this notebook

from models.dummy_model import DummyModel
from util.solver import Solver
from data.nuim_dataloader import ICLNUIMDataset
from torch.utils import data
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms
import torch
import numpy as np

In [5]:
# Check training on GPU?

cuda = torch.cuda.is_available()

print("Training is on GPU with CUDA: {}".format(cuda))

device = "cuda:0" if cuda else "cpu"

print("Device: {}".format(device))

Training is on GPU with CUDA: True
Device: cuda:0


# Load Data and Model

Load ICL-NUIM dataset.

Load the model for this notebook.

In [6]:
# Load Dataset from drive or local

if is_on_colab:
    path = "/content/drive/My Drive/Novel_View_Synthesis/ICL-NUIM/living_room_traj2_loop"
else:
    path = "/home/lukas/ICL-NUIM/prerendered_data/living_room_traj0_loop"

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])
    
data_dict = {
    "path": path,
}
    
dataset = ICLNUIMDataset(path, transform=transform)

print("Loaded following data: {} (samples: {})".format(data_dict["path"], len(dataset)))

Loaded following data: /home/lukas/ICL-NUIM/prerendered_data/living_room_traj0_loop (samples: 1510)


In [7]:
# Create Train and Val dataset with 80% train and 20% val.
# from: https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

dataset_args = {
    "batch_size": 32,
    "validation_percentage": 0.2,
    "shuffle_dataset": True,
    **data_dict
}

num_workers = 4
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(dataset_args["validation_percentage"] * dataset_size))
if dataset_args["shuffle_dataset"]:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=dataset_args["batch_size"], 
                                           sampler=train_sampler, num_workers=num_workers)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=dataset_args["batch_size"],
                                                sampler=valid_sampler, num_workers=num_workers)

dataset_args["train_len"] = len(train_loader)
dataset_args["val_len"] = len(validation_loader)

print("Dataset parameters: {}".format(dataset_args))

Dataset parameters: {'batch_size': 32, 'validation_percentage': 0.2, 'shuffle_dataset': True, 'path': '/home/lukas/ICL-NUIM/prerendered_data/living_room_traj0_loop', 'train_len': 38, 'val_len': 10}


In [8]:
# Sample loader for the purpose of this jupyter notebook: What do we want to train with the loaded dataset?
# This can be different in different jupyter notebooks, even when using the same dataset
# e.g. the feature_encoder uses NUIM dataset self-supervised, but the depth network uses it with ground-truth depth

# See the solver class: The convention is to have a dict {'x': input, 'y': target} that the solver will use to
# access the data. Therefore, we need to provide a function that returns such a dict from the dataset for each sample.

# See the ICL-NUIM dataset class: It returns a dict {'image': img, 'depth': depth} for each sample.
# Thus convert it this way.

sample_loader = lambda sample: {
    'x': sample['image'],
    'y': sample['depth']
}

In [9]:
# Load baseline model
model_args={
    "dummy_param": "bar"
}

model = DummyModel(dummy_param=model_args["dummy_param"])
model_args["model"] = type(model).__name__

print("Model configuration: {}".format(model_args))

print(model)

Model configuration: {'dummy_param': 'bar', 'model': 'DummyModel'}
DummyModel(
  (conv): Conv2d(3, 1, kernel_size=(1, 1), stride=(1, 1))
)


# Training Visualization

Start Tensorboard for visualization of the upcoming training / validation / test steps.

In [10]:
# Start tensorboard. Might need to make sure, that the correct runs directory is chosen here.
%load_ext tensorboard
%tensorboard --logdir ../runs

Reusing TensorBoard on port 6007 (pid 12130), started 0:04:55 ago. (Use '!kill 12130' to kill it.)

# Training

Start training process.

In [11]:
# Create unique ID for this training process for saving to disk.

from datetime import datetime
import uuid
now = datetime.now() # current date and time
id = str(uuid.uuid1())
id_suffix = now.strftime("%Y-%b-%d_%H-%M-%S") + "_" + id

log_dir = "../runs/DummyModel/" + id_suffix # Might need to make sure, that the correct runs directory is chosen here.
print("log_dir:", log_dir)

log_dir: ../runs/DummyModel/2020-Apr-14_10-35-23_e22d1230-7e2a-11ea-ac62-95aa6ba364ef


In [12]:
# Configure solver
extra_args = {
    **model_args,
    **dataset_args
}

solver = Solver(optim=torch.optim.Adam,
                optim_args={ "lr": 2e-4,
                             "betas": (0.9, 0.999),
                             "eps": 1e-8,
                             "weight_decay": 0.1}, # is the l2 regularization parameter, see: https://pytorch.org/docs/stable/optim.html
                #loss_func=torch.nn.CrossEntropyLoss(),
                loss_func=torch.nn.MSELoss(),
                acc_func=lambda scores,y: np.mean(np.isclose(scores.data.cpu().numpy(),y.data.cpu().numpy())),
                sample_loader=sample_loader,
                extra_args=extra_args,
                log_dir=log_dir)

Hyperparameters of this solver: {'loss function': 'MSELoss', 'optimizer': 'Adam', 'learning rate': 0.0002, 'weight_decay': 0.1, 'dummy_param': 'bar', 'model': 'DummyModel', 'batch_size': 32, 'validation_percentage': 0.2, 'shuffle_dataset': True, 'path': '/home/lukas/ICL-NUIM/prerendered_data/living_room_traj0_loop', 'train_len': 38, 'val_len': 10}


In [13]:
# Start training

solver.train(model, train_loader, validation_loader, num_epochs=1, log_nth=1)

START TRAIN on device: cuda:0


[Iteration 1/38] TRAIN loss: 5.5767316818237305
[Iteration 2/38] TRAIN loss: 6.277082443237305
[Iteration 3/38] TRAIN loss: 5.159941673278809
[Iteration 4/38] TRAIN loss: 6.326313018798828
[Iteration 5/38] TRAIN loss: 6.16605806350708
[Iteration 6/38] TRAIN loss: 5.243426322937012
[Iteration 7/38] TRAIN loss: 6.722830772399902
[Iteration 8/38] TRAIN loss: 7.893655776977539
[Iteration 9/38] TRAIN loss: 6.807187080383301
[Iteration 10/38] TRAIN loss: 6.112823963165283
[Iteration 11/38] TRAIN loss: 6.538804054260254
[Iteration 12/38] TRAIN loss: 6.660143852233887
[Iteration 13/38] TRAIN loss: 7.437685489654541
[Iteration 14/38] TRAIN loss: 5.985506534576416
[Iteration 15/38] TRAIN loss: 5.27248477935791
[Iteration 16/38] TRAIN loss: 6.764962673187256
[Iteration 17/38] TRAIN loss: 6.0676727294921875
[Iteration 18/38] TRAIN loss: 5.52291202545166
[Iteration 19/38] TRAIN loss: 5.299670696258545
[Iteration 20/38] TRAIN loss: 6.2570695877075195
[Iteration 21/38] TRAIN loss: 6.6217875480651855


[Iteration 1/10] Val loss: 5.796536445617676
[Iteration 2/10] Val loss: 6.356241226196289
[Iteration 3/10] Val loss: 6.2166523933410645
[Iteration 4/10] Val loss: 5.523354530334473
[Iteration 5/10] Val loss: 5.755112171173096
[Iteration 6/10] Val loss: 6.589200973510742
[Iteration 7/10] Val loss: 5.556537628173828
[Iteration 8/10] Val loss: 5.87794828414917
[Iteration 9/10] Val loss: 6.929519176483154
[Iteration 10/10] Val loss: 6.705647945404053

[EPOCH 1/1] VAL mean acc/loss: 0.0/6.130675315856934
FINISH.


# Test

Test with test dataset.
Will load the data and start the training.

Visualizations can be seen in Tensorboard above.

In [14]:
# Load test data
# TODO: Find real test split, for now we load the SAME dataset as for train/val (just that this notebook is complete...)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

test_path = path # CHANGE HERE TO REAL PATH TO TEST SET

test_dataset = dataset = ICLNUIMDataset(test_path, transform=transform)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=dataset_args["batch_size"], 
                                               shuffle=True,
                                               num_workers=4)

print("Length of test set: {}".format(len(test_dataset)))
print("Loaded test set: {}".format(test_path))

Length of test set: 1510
Loaded test set: /home/lukas/ICL-NUIM/prerendered_data/living_room_traj0_loop


In [15]:
# Start testing

solver.test(model, test_loader, test_prefix="DUMMY_TEST_WITH_NO_REAL_TEST_SET", log_nth=1)

[Iteration 1/48] TEST loss: 6.902449131011963
[Iteration 2/48] TEST loss: 6.080688953399658
[Iteration 3/48] TEST loss: 5.955017566680908
[Iteration 4/48] TEST loss: 5.730878829956055
[Iteration 5/48] TEST loss: 5.127932548522949
[Iteration 6/48] TEST loss: 6.427927494049072
[Iteration 7/48] TEST loss: 6.471704483032227
[Iteration 8/48] TEST loss: 6.494593143463135
[Iteration 9/48] TEST loss: 7.64017915725708
[Iteration 10/48] TEST loss: 6.741467475891113
[Iteration 11/48] TEST loss: 5.534629821777344
[Iteration 12/48] TEST loss: 6.85181999206543
[Iteration 13/48] TEST loss: 5.439666271209717
[Iteration 14/48] TEST loss: 5.621309757232666
[Iteration 15/48] TEST loss: 6.6567277908325195
[Iteration 16/48] TEST loss: 6.286664009094238
[Iteration 17/48] TEST loss: 6.500030040740967
[Iteration 18/48] TEST loss: 5.912247657775879
[Iteration 19/48] TEST loss: 7.012545585632324
[Iteration 20/48] TEST loss: 6.0244317054748535
[Iteration 21/48] TEST loss: 5.35226583480835
[Iteration 22/48] TEST 

# Save the model

Save network with its weights to disk.

See torch.save function: https://pytorch.org/docs/stable/notes/serialization.html#recommend-saving-models 

Load again with `the_model = TheModelClass(*args, **kwargs) the_model.load_state_dict(torch.load(PATH))`

In [16]:
def save_model(modelname, model):
    # Might need to make sure, that the correct saved_results directory is chosen here.
    filepath = "../saved_models/" + modelname + ".pt"
    torch.save(model.state_dict(), filepath)

In [17]:
modelname = "dummy_" + id_suffix
save_model(modelname, model)

In [18]:
# LOAD MODEL AGAIN for verification purposes
# Should print: <All keys matched successfully>

filepath = "../saved_models/" + modelname + ".pt"
model.load_state_dict(torch.load(filepath))

<All keys matched successfully>